<a href="https://colab.research.google.com/github/yudhiesh1997/Tensorflow-Tutorial/blob/master/Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("test.csv", TEST_DATA_URL)


16384/13049 [=====================================] - 0s 0us/step


In [0]:
# Make numpy values easier to read
np.set_printoptions(precision=3,suppress=True)

In [6]:
# Load data 
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0,1]


In [0]:
# Now read the CSV from the file and create a dataset
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size = 5,
      label_name = LABEL_COLUMN,
      na_value = "?",
      num_epochs = 1,
      ignore_errors =True,
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)



In [0]:
# Each item in the dataset is a batch 
# Which is represented by a tuple 
# 
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [13]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [29. 16. 35. 28. 28.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [ 30.     26.    512.329  19.967   9.5  ]
class               : [b'First' b'Second' b'First' b'Third' b'Third']
deck                : [b'D' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [14]:
# If dataset does not contain the name of the column 
# pass the names in 
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [30. 27. 16. 35. 32.]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 3 0 0]
fare                : [12.475 10.5   34.375 26.     7.75 ]
class               : [b'Third' b'Second' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'E' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [15]:
# If dataset requires that you omit some of the columns 
# Just create a new column with the required names

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [ 9. 27. 28. 20. 31.]
n_siblings_spouses  : [5 1 0 0 0]
class               : [b'Third' b'Second' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'C']
alone               : [b'n' b'n' b'y' b'y' b'n']


In [0]:
# Data preprocessinng 
# Data comes in a mixed form and you want to convert them to a fixed length vector
# before sending it for training to your Neural Network 



In [16]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [41. 57. 18. 32. 30.]
n_siblings_spouses  : [0. 0. 0. 0. 3.]
parch               : [0. 0. 2. 0. 0.]
fare                : [ 7.125 12.35  13.    13.    21.   ]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
# Function to pack the columns 
def pack(features, label):
  return tf.stack(list(features.values()), axis = -1), label 

In [19]:
# Apply this to each element of the dataset
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.     0.     0.     7.733]
 [28.     2.     0.    23.25 ]
 [28.     1.     0.    15.5  ]
 [49.     1.     0.    76.729]
 [28.     3.     1.    25.467]]

[1 1 1 1 0]


In [21]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [33. 19.  3.  9. 28.]
n_siblings_spouses  : [0 0 1 3 0]
parch               : [2 0 1 2 1]
fare                : [26.    10.171 18.75  27.9   33.   ]
class               : [b'Second' b'Third' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'n' b'n' b'n']


In [0]:
example_patch, labels_batch = next(iter(temp_dataset))

In [0]:
# More general preprocessor
# Selects a list of numeric features and packs them into a single column

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis = -1)
    features['numeric'] = numeric_features

    return features, labels 


In [0]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES)
)
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))


In [26]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'male']
class               : [b'Third' b'First' b'Second' b'First' b'Second']
deck                : [b'G' b'D' b'unknown' b'B' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'y' b'y']
numeric             : [[ 2.     0.     1.    10.462]
 [48.     1.     0.    76.729]
 [24.     0.     2.    14.5  ]
 [28.     0.     0.     0.   ]
 [23.     0.     0.    13.   ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data))

In [30]:
# Data normalization

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data 
  return (data-mean)/std

In [34]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f085a3ee8c8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [35]:
# Whenn training the model include this feature colum 
# Also center the block of numeric data 
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 27.   ,   0.   ,   0.   ,   8.663],
       [ 36.   ,   0.   ,   0.   ,  26.388],
       [ 28.   ,   0.   ,   0.   ,  26.55 ],
       [  4.   ,   4.   ,   2.   ,  31.275],
       [ 24.   ,   0.   ,   1.   , 247.521]], dtype=float32)>

In [37]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[-0.21 , -0.474, -0.479, -0.471],
       [ 0.509, -0.474, -0.479, -0.146],
       [-0.13 , -0.474, -0.479, -0.144],
       [-2.049,  3.001,  2.043, -0.057],
       [-0.45 , -0.474,  0.782,  3.904]], dtype=float32)

In [0]:
# Categorical data 
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col
                                                                 ))

In [41]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [44]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [0]:
# Add the 2 feature column collections 
# Pass them into DenseFeatures to create an input layer 
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

In [46]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.21  -0.474
 -0.479 -0.471  1.     0.   ]


In [0]:
# Build the model 
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),

])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits= True),
               optimizer='adam',
               metrics=['accuracy'])


In [0]:
# Train, Evaluat and predict 
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [53]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 11ms/step - loss: 0.5032 - accuracy: 0.7448
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4199 - accuracy: 0.8278
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4029 - accuracy: 0.8325
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3824 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3774 - accuracy: 0.8325
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3664 - accuracy: 0.8389
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3674 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3647 - accuracy: 0.8405
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3443 - accuracy: 0.8453
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3543 - accuracy: 0.840

In [55]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))


     53/Unknown - 0s 7ms/step - loss: 0.4678 - accuracy: 0.8485

Test Loss 0.4678055078212945, Test Accuracy 0.8484848737716675


In [58]:
predictions = model.predict(test_data)

# Show some results 
for predictions, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2f}".format(predictions[0]),
        "Actual Outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: -0.99 Actual Outcome:  DIED
Predicted survival: -0.30 Actual Outcome:  DIED
Predicted survival: -2.48 Actual Outcome:  SURVIVED
Predicted survival: -1.68 Actual Outcome:  DIED
Predicted survival: 0.35 Actual Outcome:  DIED
